# Recruiting data analysis

In this notebook we do some basic analysis of the synthetic recruiting data, to understand existing biases before training a model.

In [ ]:
from pathlib import Path

import pandas as pd
from helpers.recruiting import bin_years_experience
from helpers.plot import bar_chart

In [ ]:
from helpers import export_plot

Directory for saving all processed data.

In [ ]:
artifacts_dir = Path("../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../artifacts")

Load the preprocessed data. Check out the preprocessing notebook for details on how this data was obtained.

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

test = pd.read_csv(data_dir / "processed" / "test.csv")
test_raw = pd.read_csv(data_dir / "raw" / "test.csv")

## Bias in data

Visualise biases present in the data

In [ ]:
employed_by_sex = test[["sex_male", "employed_yes"]].groupby("sex_male").mean()

fig_employed_by_sex = bar_chart(
    x=["Female", "Male"],
    y=employed_by_sex.employed_yes,
    title="Propotion of applicants employed by sex",
    ylabel="Proportion",
)
fig_employed_by_sex

In [ ]:
export_plot(fig_employed_by_sex, "employed_by_sex.json")

In [ ]:
employed_by_race = (
    test[["race_white", "employed_yes"]].groupby("race_white").mean()
)

fig_employed_by_race = bar_chart(
    x=["Black", "White"],
    y=employed_by_race.employed_yes,
    title="Propotion of applicants employed by race",
    ylabel="Proportion",
)
fig_employed_by_race

In [ ]:
export_plot(fig_employed_by_race, "employed_by_race.json")

### Possible resolving variables

Let's look at the relationship between years of experience and whether the applicant was employed.

In [ ]:
employed_by_experience = (
    test.assign(exp=test_raw.years_experience.map(bin_years_experience))
    .loc[:, ["exp", "employed_yes"]]
    .groupby("exp")
    .aggregate(["mean", "count"])
    .reset_index()
)

fig_employed_by_experience = bar_chart(
    x=employed_by_experience.exp,
    y=employed_by_experience["employed_yes"]["mean"],
    title="Proportion of applicants employed by years of experience",
    ylabel="Proportion",
    xticks={
        "tickvals": [0, 1, 2, 3],
        "ticktext": ["0-2 years", "3-5 years", "6-9 years", "10+ years"],
    },
)
fig_employed_by_experience

In [ ]:
export_plot(
    fig_employed_by_experience, "employed_by_experience.json",
)